In [ ]:
import polars as pl
import plotly.express as px

### Pre každý deň by sme chceli znázorniť grafy po hodinách. Urobíme pre nástupy

In [ ]:
# nacitame vycistenu frejmu, len stlpce, co pouzijeme v tomto NB
cols = ['pick_dt', 'passengers', 'fare']
df = pl.scan_parquet('data/nyc_taxi310k.parq')
df = df.select(cols).collect()   

In [ ]:
# Potrebujeme zoskupit data podla dni a hodin; groupped sama o sebe je nam netreba
groupped = df.group_by(pl.col('pick_dt').dt.day().alias('pick_day'), 
                       pl.col('pick_dt').dt.hour().alias('pick_hour'))
type(groupped)

In [ ]:
# groupped nie je datafrejma, ale GroupBy, z nej vela osohu nie je, no hodi sa na agregaciu
# to dole je len pre zaujimavost, ako je GroupBy zostrojena
kolko = 3
for name, data in groupped:  
    print(name)
    print(data)
    kolko -= 1
    if not kolko:
        break

In [ ]:
df_days = groupped.agg([pl.col('passengers').sum().alias('pass_count'),
                        pl.col('fare').count().alias('fares_count'),
                        pl.col('fare').sum().alias('total_fare')])

In [ ]:
df_days.sort(by='pick_day').head()

In [ ]:
# ako vyberieme jeden den? lahko, napr. 14. jan.
df_days.filter(pl.col('pick_day') == 14).sort(by='pick_hour').head()
df_days.columns

In [ ]:
# funkcia pre vyrobenie df_days z povodnej df; urobime grupovanie a agregacie na jeden sup
def daily_frame(frm):   # vyvolame s df ako frm
    df_days = frm.group_by(pl.col('pick_dt').dt.day().alias('pick_day'), 
                          pl.col('pick_dt').dt.hour().alias('pick_hour'))\
                         .agg([pl.col('passengers').sum().alias('pass_count'),
                               pl.col('fare').count().alias('fares_count'),
                               pl.col('fare').sum().alias('total_fare')])
    return df_days

In [ ]:
# mame vsetko, aby sme kreslili, podobne bude ako monthly_plot z data_funkcie
def daily_plot(day): # frm tu bude povyssia df_days
    frm_d = daily_frame(df)
    frm_day = frm_d.filter(pl.col('pick_day') == day).sort(by='pick_hour')
    pass_fares_plot = px.bar(frm_day, x='pick_hour', y=['pass_count', 'fares_count'], barmode='group',
                             labels={'pick_hour': 'Hodina', 'value': 'Hodnoty', 'variable': 'Premenná'})
    pass_fares_plot.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(24))))
    return pass_fares_plot

In [ ]:
daily_plot(26)

#### Obe funkcie, daily_frame aj daily_plot nakopčíme do `data_funkcie.py` pre budúce užitie